In [7]:
# Check NumPy version
import numpy as np
print("NumPy version:", np.__version__)

NumPy version: 1.26.4


In [8]:
# Import libraries for data manipulation and SVD
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

In [9]:
# Load the processed dataset
df = pd.read_csv('../data/processed/stress_relief_data.csv')
# Verify columns and first few rows
print('Columns:', df.columns.tolist())
print('First 5 rows:\n', df.head())

Columns: ['user_id', 'mood', 'stress_level', 'activity', 'feedback']
First 5 rows:
    user_id     mood stress_level    activity  feedback
0     1999     calm          low  journaling         0
1     2631  anxious         high     walking         1
2      446  anxious         high       music         1
3      517  anxious         high  meditation         0
4     2760     calm          low   breathing         1


In [10]:
# Prepare data for Surprise
# Define the rating scale (0=not liked, 1=liked)
reader = Reader(rating_scale=(0, 1))
# Load data into Surprise format (user_id, activity, feedback)
data = Dataset.load_from_df(df[['user_id', 'activity', 'feedback']], reader)
# Split into train and test sets (80% train, 20% test)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
print('Training set size:', trainset.n_ratings)

Training set size: 4444


In [11]:
# Train SVD model
model = SVD(n_factors=10, random_state=42)
model.fit(trainset)
# Test the model on the test set
predictions = model.test(testset)
# Calculate RMSE
rmse = accuracy.rmse(predictions)
print(f'RMSE on test set: {rmse:.3f}')

RMSE: 0.4578
RMSE on test set: 0.458


In [12]:
# Function to recommend activities for a user
def recommend_activities(user_id, model, activities, n=3):
    # Get all activities
    predictions = []
    for activity in activities:
        pred = model.predict(user_id, activity)
        predictions.append((activity, pred.est))
    # Sort by predicted rating and return top n
    predictions.sort(key=lambda x: x[1], reverse=True)
    return [activity for activity, _ in predictions[:n]]

# Test recommendations for a sample user
activities = ['meditation', 'music', 'journaling', 'breathing', 'walking']
sample_user_id = 1
recommended = recommend_activities(sample_user_id, model, activities)
print(f'Recommended activities for user {sample_user_id}:', recommended)

Recommended activities for user 1: ['walking', 'journaling', 'music']
